In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import os
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score

In [10]:
# Plotting style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_context("talk")
colors = sns.color_palette("viridis", 10)  # One color for each year

In [11]:
# Create directories for outputs
os.makedirs('figures', exist_ok=True)
os.makedirs('reports', exist_ok=True)
os.makedirs('models', exist_ok=True)

## 1. Data Loading Functions

In [5]:
def load_statcast_data(year):
    """Load Statcast data for a specific year"""
    filename = f'data/statcast-{year}.csv'
    df = pd.read_csv(filename)
    df['Year'] = year
    
    # Handle potential string formatting issues with percentages
    for col in ['xBA', 'xSLG', 'xwOBA']:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = df[col].astype(float)
    
    return df

def load_standard_data(year):
    """Load standard batting data for a specific year"""
    filename = f'data/standard-{year}.csv'
    df = pd.read_csv(filename)
    df['Year'] = year
    return df

def merge_yearly_data(statcast_df, standard_df):
    """Merge Statcast and standard data for a given year"""
    # Ensure Team columns are formatted consistently
    statcast_df['Team'] = statcast_df['Team'].str.strip()
    standard_df['Team'] = standard_df['Team'].str.strip()
    
    # Merge on Team and Year
    merged = pd.merge(statcast_df, standard_df, on=['Team', 'Year'], suffixes=('_statcast', ''))
    
    # If there are duplicate PA columns, use the one from standard data
    if 'PA_statcast' in merged.columns:
        merged = merged.drop('PA_statcast', axis=1)
    
    return merged

## 2. Load All Data (2015-2024)

In [12]:
years = range(2015, 2025)
all_data = []

for year in years:
    try:
        statcast = load_statcast_data(year)
        standard = load_standard_data(year)
        merged = merge_yearly_data(statcast, standard)
        all_data.append(merged)
        print(f"Successfully loaded data for {year}")
    except Exception as e:
        print(f"Error loading data for {year}: {e}")
        continue

# Combine all years into one dataframe
combined_df = pd.concat(all_data, ignore_index=True)

Successfully loaded data for 2015
Successfully loaded data for 2016
Successfully loaded data for 2017
Successfully loaded data for 2018
Successfully loaded data for 2019
Successfully loaded data for 2020
Successfully loaded data for 2021
Successfully loaded data for 2022
Successfully loaded data for 2023
Successfully loaded data for 2024


## 3. Data Cleaning and Preprocessing

In [13]:
def clean_data(df):
    # Check for missing values
    missing_values = df.isnull().sum()
    print("Missing values per column:")
    print(missing_values[missing_values > 0])
    
    # Handle missing values - for numeric columns, fill with median
    numeric_cols = df.select_dtypes(include=[np.number]).columns
    for col in numeric_cols:
        if df[col].isnull().sum() > 0:
            df[col] = df[col].fillna(df[col].median())
    
    # Create additional metrics
    if 'OBP' not in df.columns and all(col in df.columns for col in ['H', 'BB', 'HBP', 'AB', 'SF']):
        df['OBP'] = (df['H'] + df['BB'] + df['HBP']) / (df['AB'] + df['BB'] + df['HBP'] + df['SF'])
    
    # Create combined power-contact metric 
    if all(col in df.columns for col in ['EV', 'Barrel%']):
        # Standardize values first
        scaler = StandardScaler()
        df['Power_Contact'] = scaler.fit_transform(df[['EV']]) * 0.6 + scaler.fit_transform(df[['Barrel%']]) * 0.4
    
    # Calculate runs per game
    if all(col in df.columns for col in ['R', 'G']):
        df['R_per_G'] = df['R'] / df['G']
    
    # Calculate strikeout rate and walk rate
    if all(col in df.columns for col in ['SO', 'BB', 'PA']):
        df['K_rate'] = df['SO'] / df['PA']
        df['BB_rate'] = df['BB'] / df['PA']
    
    return df

# Clean the data
combined_df = clean_data(combined_df)

# Create yearly dataframes dictionary for separate analyses
yearly_dfs = {year: combined_df[combined_df['Year'] == year] for year in years if year in combined_df['Year'].unique()}

Missing values per column:
xBA      300
xSLG     300
xwOBA    300
dtype: int64


## 4. Exploratory Data Analysis and Visualization

In [14]:
def plot_run_distribution_by_year(df):
    """Plot run distribution for each year"""
    plt.figure(figsize=(14, 8))
    sns.boxplot(x='Year', y='R', data=df, palette="viridis")
    plt.title('Run Distribution by Year', fontsize=18)
    plt.xlabel('Year', fontsize=14)
    plt.ylabel('Runs Scored', fontsize=14)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('figures/run_distribution_by_year.png', dpi=300)
    plt.close()

def plot_metric_evolution(df, metrics):
    """Plot how metrics evolved over the years"""
    plt.figure(figsize=(15, 10))
    
    for i, metric in enumerate(metrics):
        if metric in df.columns:
            yearly_means = df.groupby('Year')[metric].mean()
            plt.plot(yearly_means.index, yearly_means.values, marker='o', linewidth=2, 
                    label=metric, color=colors[i % len(colors)])
    
    plt.title('Evolution of Batting Metrics (2015-2024)', fontsize=18)
    plt.xlabel('Year', fontsize=14)
    plt.ylabel('Average Value', fontsize=14)
    plt.legend(fontsize=12)
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('figures/metric_evolution.png', dpi=300)
    plt.close()

def plot_correlation_heatmap(df, year=None):
    """Plot correlation heatmap between batting metrics and run production"""
    metrics = ['AVG', 'OBP', 'SLG', 'HR', 'BB', 'SO', 'EV', 'LA', 'Barrel%', 'HardHit%', 'R']
    available_metrics = [m for m in metrics if m in df.columns]
    
    # Calculate correlation matrix
    corr = df[available_metrics].corr()
    
    plt.figure(figsize=(12, 10))
    mask = np.triu(np.ones_like(corr, dtype=bool))
    sns.heatmap(corr, mask=mask, cmap='coolwarm', annot=True, fmt='.2f', linewidths=0.5,
                vmin=-1, vmax=1, center=0, square=True)
    
    year_text = f" - {year}" if year else " - All Years"
    plt.title(f'Correlation Between Batting Metrics{year_text}', fontsize=18)
    plt.tight_layout()
    filename = f'figures/correlation_heatmap{"_"+str(year) if year else "_all_years"}.png'
    plt.savefig(filename, dpi=300)
    plt.close()

def plot_run_correlation_by_year(df, metrics):
    """Plot correlation of metrics with runs over time"""
    years = sorted(df['Year'].unique())
    correlations = {}
    
    for metric in metrics:
        if metric in df.columns:
            correlations[metric] = []
            for year in years:
                year_df = df[df['Year'] == year]
                if len(year_df) > 5:
                    corr = year_df[metric].corr(year_df['R'])
                    correlations[metric].append(corr)
                else:
                    correlations[metric].append(np.nan)
    
    plt.figure(figsize=(14, 8))
    for i, (metric, corrs) in enumerate(correlations.items()):
        plt.plot(years, corrs, marker='o', linewidth=2, label=metric, color=colors[i % len(colors)])
    
    plt.title('Correlation with Run Production Over Time', fontsize=18)
    plt.xlabel('Year', fontsize=14)
    plt.ylabel('Correlation Coefficient (r)', fontsize=14)
    plt.axhline(y=0, color='gray', linestyle='--', alpha=0.7)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('figures/run_correlation_by_year.png', dpi=300)
    plt.close()

def scatter_plots_with_runs(df, metrics, year=None):
    """Create scatter plots of each metric vs runs"""
    n_metrics = len(metrics)
    n_cols = 3
    n_rows = (n_metrics + n_cols - 1) // n_cols
    
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(18, 4*n_rows))
    axes = axes.flatten()
    
    if year is not None:
        plot_df = df[df['Year'] == year]
    else:
        plot_df = df
    
    for i, metric in enumerate(metrics):
        if metric in df.columns and i < len(axes):
            ax = axes[i]
            sns.regplot(x=metric, y='R', data=plot_df, ax=ax, scatter_kws={'alpha':0.7}, 
                       line_kws={'color':'red'})
            
            # Calculate and show correlation coefficient
            if len(plot_df) > 1: 
                corr, p_value = stats.pearsonr(plot_df[metric].dropna(), plot_df['R'].dropna())
                ax.text(0.05, 0.95, f'r = {corr:.3f}\np = {p_value:.3f}', 
                        transform=ax.transAxes, fontsize=12,
                        verticalalignment='top')
            
            ax.set_title(f'{metric} vs Runs')
    
    # Hide any unused subplots
    for j in range(i+1, len(axes)):
        axes[j].set_visible(False)
    
    year_text = f" - {year}" if year else " - All Years"
    plt.suptitle(f'Relationship Between Batting Metrics and Run Production{year_text}', fontsize=20)
    plt.tight_layout(rect=[0, 0, 1, 0.97])
    filename = f'figures/run_scatter_plots{"_"+str(year) if year else "_all_years"}.png'
    plt.savefig(filename, dpi=300)
    plt.close()

# Run EDA visualizations
key_metrics = ['AVG', 'OBP', 'SLG', 'HR', 'BB', 'BB_rate', 'K_rate', 'EV', 'LA', 'Barrel%', 'HardHit%']
available_metrics = [m for m in key_metrics if m in combined_df.columns]

# Generate plots for all years combined
plot_run_distribution_by_year(combined_df)
plot_metric_evolution(combined_df, available_metrics)
plot_correlation_heatmap(combined_df)
plot_run_correlation_by_year(combined_df, available_metrics)
scatter_plots_with_runs(combined_df, available_metrics)

# Generate plots for each year
for year, df in yearly_dfs.items():
    if len(df) > 5:  # Only generate if enough data points
        plot_correlation_heatmap(df, year)
        scatter_plots_with_runs(df, available_metrics, year)

## 5. Statistical Modeling

In [15]:
def run_regression_models(df, year=None):
    """Run regression models to predict run production"""
    potential_predictors = ['AVG', 'OBP', 'SLG', 'HR', 'BB', 'BB_rate', 'K_rate', 
                           'EV', 'LA', 'Barrel%', 'HardHit%']
    
    # Filter available predictors
    available_predictors = [p for p in potential_predictors if p in df.columns]
    model_df = df[df['Year'] == year] if year else df
    
    # Check for multicollinearity
    if len(model_df) > len(available_predictors) + 5: 
        correlation_matrix = model_df[available_predictors].corr()
        high_correlation_pairs = []
        
        for i in range(len(correlation_matrix.columns)):
            for j in range(i+1, len(correlation_matrix.columns)):
                if abs(correlation_matrix.iloc[i, j]) > 0.7:
                    high_correlation_pairs.append((correlation_matrix.columns[i], 
                                                correlation_matrix.columns[j], 
                                                correlation_matrix.iloc[i, j]))
        
        print(f"High correlation pairs{' for year '+str(year) if year else ''}:")
        for pair in high_correlation_pairs:
            print(f"{pair[0]} and {pair[1]}: {pair[2]:.3f}")
        
        # Create different model groups to avoid multicollinearity
        traditional_metrics = [p for p in ['AVG', 'OBP', 'SLG', 'HR', 'BB'] if p in available_predictors]
        statcast_metrics = [p for p in ['EV', 'LA', 'Barrel%', 'HardHit%'] if p in available_predictors]
        rate_metrics = [p for p in ['BB_rate', 'K_rate'] if p in available_predictors]
        
        # Run models for different metric groups
        model_groups = {
            'Traditional': traditional_metrics,
            'Statcast': statcast_metrics,
            'Combined': traditional_metrics[:2] + statcast_metrics[:2] + rate_metrics  # Select top metrics from each
        }
        
        results = {}
        
        for group_name, predictors in model_groups.items():
            if len(predictors) >= 1:
                # Prepare data
                X = model_df[predictors].dropna()
                y = model_df.loc[X.index, 'R']
                
                if len(X) > len(predictors) + 5: 
                    # Add constant for statsmodels
                    X_sm = sm.add_constant(X)
                    
                    # Fit OLS model
                    model = sm.OLS(y, X_sm).fit()
                    
                    # Store results
                    results[group_name] = {
                        'model': model,
                        'summary': model.summary(),
                        'r_squared': model.rsquared,
                        'adj_r_squared': model.rsquared_adj,
                        'predictors': predictors
                    }
                    
                    print(f"\n{group_name} Model{' for year '+str(year) if year else ''}:")
                    print(f"R-squared: {model.rsquared:.4f}")
                    print(f"Adjusted R-squared: {model.rsquared_adj:.4f}")
                    print("Significant predictors:")
                    
                    for var, p_value in zip(model.params.index[1:], model.pvalues[1:]):
                        if p_value < 0.05:
                            print(f"- {var}: coefficient = {model.params[var]:.4f}, p-value = {p_value:.4f}")
        
        return results
    else:
        print(f"Not enough data points for regression analysis{' for year '+str(year) if year else ''}")
        return None

# Run regression models for all years combined
combined_results = run_regression_models(combined_df)

# Run regression for each year
yearly_models = {}
for year, df in yearly_dfs.items():
    yearly_models[year] = run_regression_models(df, year)

High correlation pairs:
AVG and OBP: 0.745
AVG and SLG: 0.713
OBP and SLG: 0.811
HR and BB: 0.798
EV and Barrel%: 0.710
EV and HardHit%: 0.868
Barrel% and HardHit%: 0.890

Traditional Model:
R-squared: 0.9553
Adjusted R-squared: 0.9546
Significant predictors:
- AVG: coefficient = 5972.6417, p-value = 0.0000
- OBP: coefficient = -2486.6579, p-value = 0.0000
- SLG: coefficient = -866.5239, p-value = 0.0019
- HR: coefficient = 1.4478, p-value = 0.0000
- BB: coefficient = 0.7603, p-value = 0.0000

Statcast Model:
R-squared: 0.0914
Adjusted R-squared: 0.0791
Significant predictors:
- EV: coefficient = 99.8404, p-value = 0.0000

Combined Model:
R-squared: 0.2134
Adjusted R-squared: 0.1973
Significant predictors:
- AVG: coefficient = 9908.8632, p-value = 0.0042
- EV: coefficient = 26.8873, p-value = 0.0131
- LA: coefficient = 12.1782, p-value = 0.0308
High correlation pairs for year 2015:
SLG and HR: 0.809
SLG and Barrel%: 0.706
HR and Barrel%: 0.810
BB and BB_rate: 0.995
EV and Barrel%: 0.79

## 6. Model Visualization and Comparison

In [16]:
def plot_model_evolution(yearly_models):
    """Plot how model performance evolved over the years"""
    years = sorted(yearly_models.keys())
    r_squared_values = {
        'Traditional': [],
        'Statcast': [],
        'Combined': []
    }
    
    for year in years:
        model_results = yearly_models[year]
        if model_results:
            for model_type in r_squared_values.keys():
                if model_type in model_results:
                    r_squared_values[model_type].append(model_results[model_type]['r_squared'])
                else:
                    r_squared_values[model_type].append(np.nan)
    
    plt.figure(figsize=(14, 8))
    for i, (model_type, values) in enumerate(r_squared_values.items()):
        plt.plot(years, values, marker='o', linewidth=2, label=model_type, color=colors[i])
    
    plt.title('Model Performance Evolution (2015-2024)', fontsize=18)
    plt.xlabel('Year', fontsize=14)
    plt.ylabel('R-squared', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('figures/model_performance_evolution.png', dpi=300)
    plt.close()

def plot_coefficient_comparison(yearly_models):
    """Plot how coefficients for key metrics changed over time"""
    years = sorted(yearly_models.keys())
    
    # Find common predictors across years
    all_predictors = set()
    for year, models in yearly_models.items():
        if models and 'Combined' in models:
            all_predictors.update(models['Combined']['predictors'])
    
    key_predictors = ['OBP', 'SLG', 'Barrel%', 'EV']
    available_predictors = [p for p in key_predictors if p in all_predictors]
    
    if not available_predictors:
        print("No common predictors found across years for coefficient comparison")
        return
    
    coefficients = {predictor: [] for predictor in available_predictors}
    
    for year in years:
        models = yearly_models[year]
        if models and 'Combined' in models:
            model = models['Combined']['model']
            for predictor in available_predictors:
                if predictor in model.params.index:
                    coefficients[predictor].append(model.params[predictor])
                else:
                    coefficients[predictor].append(np.nan)
    
    plt.figure(figsize=(14, 8))
    for i, (predictor, values) in enumerate(coefficients.items()):
        plt.plot(years, values, marker='o', linewidth=2, label=predictor, color=colors[i])
    
    plt.title('Regression Coefficients for Key Metrics Over Time', fontsize=18)
    plt.xlabel('Year', fontsize=14)
    plt.ylabel('Coefficient Value', fontsize=14)
    plt.grid(True, alpha=0.3)
    plt.legend(fontsize=12)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.savefig('figures/coefficient_evolution.png', dpi=300)
    plt.close()

# Visualize model results
if all(bool(yearly_models[year]) for year in yearly_models if year in combined_df['Year'].unique()):
    plot_model_evolution(yearly_models)
    plot_coefficient_comparison(yearly_models)

## 7. Identify Team Patterns and Strategies

In [17]:
def analyze_team_strategies(df):
    """Analyze team offensive strategies and clustering"""
    # Create metrics for different offensive styles
    if all(col in df.columns for col in ['SLG', 'HR', 'EV', 'Barrel%']):
        power_metrics = ['SLG', 'HR', 'EV', 'Barrel%']
        available_power = [m for m in power_metrics if m in df.columns]
        
        # Standardize values
        scaler = StandardScaler()
        power_values = scaler.fit_transform(df[available_power])
        df['Power_Score'] = np.mean(power_values, axis=1)
    
    if all(col in df.columns for col in ['AVG', 'OBP', 'BB', 'K_rate']):
        contact_metrics = ['AVG', 'OBP', 'BB']
        available_contact = [m for m in contact_metrics if m in df.columns]
        
        # For contact score, BB has positive impact, K_rate negative
        contact_values = scaler.fit_transform(df[available_contact])
        if 'K_rate' in df.columns:
            k_rate_std = -1 * scaler.fit_transform(df[['K_rate']])
            contact_values = np.hstack((contact_values, k_rate_std))
            
        df['Contact_Score'] = np.mean(contact_values, axis=1)
    
    # Plot team strategies
    if 'Power_Score' in df.columns and 'Contact_Score' in df.columns:
        plt.figure(figsize=(14, 10))
        
        # Create runs per game size scale for scatter points
        if 'R_per_G' in df.columns:
            size_scale = 50 * (df['R_per_G'] - df['R_per_G'].min()) / (df['R_per_G'].max() - df['R_per_G'].min()) + 30
        else:
            size_scale = 50
        
        # Color by year
        year_norm = (df['Year'] - df['Year'].min()) / (df['Year'].max() - df['Year'].min())
        scatter = plt.scatter(df['Contact_Score'], df['Power_Score'], 
                             s=size_scale, c=year_norm, cmap='viridis', alpha=0.7, 
                             edgecolor='k', linewidth=0.5)
        
        # Add team labels
        for i, txt in enumerate(df['Team']):
            plt.annotate(f"{txt}-{df['Year'].iloc[i]}", 
                        (df['Contact_Score'].iloc[i], df['Power_Score'].iloc[i]),
                        fontsize=8, alpha=0.7)
        
        plt.axhline(y=0, color='gray', linestyle='--', alpha=0.5)
        plt.axvline(x=0, color='gray', linestyle='--', alpha=0.5)
        
        plt.title('Team Offensive Strategies (2015-2024)', fontsize=18)
        plt.xlabel('Contact Approach', fontsize=14)
        plt.ylabel('Power Approach', fontsize=14)
        
        # Add colorbar for years
        cbar = plt.colorbar(scatter)
        cbar.set_label('Year', fontsize=12)
        
        # Add size legend for runs per game
        if 'R_per_G' in df.columns:
            run_ranges = np.linspace(df['R_per_G'].min(), df['R_per_G'].max(), 4)
            sizes = 50 * (run_ranges - df['R_per_G'].min()) / (df['R_per_G'].max() - df['R_per_G'].min()) + 30
            
            handles, labels = [], []
            for i, run_val in enumerate(run_ranges):
                handles.append(plt.scatter([], [], s=sizes[i], color='gray', alpha=0.7, edgecolor='k'))
                labels.append(f'{run_val:.2f} R/G')
            
            plt.legend(handles, labels, title="Runs per Game", 
                     loc="upper left", bbox_to_anchor=(1, 1))
        
        plt.tight_layout()
        plt.savefig('figures/team_strategies.png', dpi=300)
        plt.close()
        
        # Find top performing teams in each quadrant
        df['Quadrant'] = pd.cut(df['Contact_Score'], bins=[-float('inf'), 0, float('inf')], labels=['Low', 'High']).astype(str) + '-' + \
                        pd.cut(df['Power_Score'], bins=[-float('inf'), 0, float('inf')], labels=['Low', 'High']).astype(str)
        
        quadrant_performance = df.groupby(['Quadrant', 'Year', 'Team'])['R'].mean().reset_index()
        top_teams = quadrant_performance.sort_values(['Quadrant', 'R'], ascending=[True, False]).groupby('Quadrant').head(5)
        
        print("\nTop performing teams by offensive strategy:")
        for quadrant, teams in top_teams.groupby('Quadrant'):
            print(f"\n{quadrant} Contact-Power Quadrant:")
            for _, row in teams.iterrows():
                print(f"- {row['Team']} ({row['Year']}): {row['R']} runs")

# Run team strategy analysis
analyze_team_strategies(combined_df)


Top performing teams by offensive strategy:

High-High Contact-Power Quadrant:
- ATL (2023): 947.0 runs
- NYY (2019): 943.0 runs
- MIN (2019): 939.0 runs
- HOU (2019): 920.0 runs
- LAD (2023): 906.0 runs

High-Low Contact-Power Quadrant:
- COL (2017): 824.0 runs
- MIN (2017): 815.0 runs
- HOU (2018): 797.0 runs
- BOS (2017): 785.0 runs
- MIA (2017): 778.0 runs

Low-High Contact-Power Quadrant:
- TBR (2021): 857.0 runs
- TEX (2019): 810.0 runs
- TEX (2017): 799.0 runs
- ATL (2021): 790.0 runs
- ATL (2022): 789.0 runs

Low-Low Contact-Power Quadrant:
- COL (2021): 739.0 runs
- TEX (2018): 737.0 runs
- MIL (2017): 732.0 runs
- BAL (2019): 729.0 runs
- MIL (2023): 728.0 runs


## 8. Summarize Findings

In [18]:
print("\n\nSUMMARY OF FINDINGS")
print("===================")
print("\n1. Correlation Analysis:")
if 'R' in combined_df.columns:
    print("\nStrongest correlations with run production (All Years):")
    correlations = [(col, combined_df[col].corr(combined_df['R'])) 
                   for col in available_metrics if col in combined_df.columns]
    correlations.sort(key=lambda x: abs(x[1]), reverse=True)
    
    for metric, corr in correlations[:5]:
        print(f"- {metric}: r = {corr:.3f}")

print("\n2. Regression Models:")
if combined_results:
    for model_type, results in combined_results.items():
        print(f"\n{model_type} Model (All Years):")
        print(f"- R-squared: {results['r_squared']:.3f}")
        print(f"- Adjusted R-squared: {results['adj_r_squared']:.3f}")
        print("- Key predictors:")
        
        significant_predictors = [(var, results['model'].params[var], results['model'].pvalues[var]) 
                                for var in results['model'].params.index[1:] 
                                if results['model'].pvalues[var] < 0.05]
        significant_predictors.sort(key=lambda x: abs(x[1]), reverse=True)
        
        for predictor, coef, p_value in significant_predictors[:3]:
            print(f"  * {predictor}: coefficient = {coef:.3f}, p-value = {p_value:.3f}")

print("\n3. Evolution of Metrics:")
if len(combined_df['Year'].unique()) > 1:
    for metric in available_metrics[:5]:  # Top 5 metrics
        if metric in combined_df.columns:
            yearly_means = combined_df.groupby('Year')[metric].mean()
            start_val = yearly_means.iloc[0]
            end_val = yearly_means.iloc[-1]
            change_pct = 100 * (end_val - start_val) / start_val
            
            print(f"- {metric}: {change_pct:+.1f}% change from {combined_df['Year'].min()} to {combined_df['Year'].max()}")

print("\n4. Team Strategy Effectiveness:")
if 'Contact_Score' in combined_df.columns and 'Power_Score' in combined_df.columns and 'R' in combined_df.columns:
    quadrant_runs = combined_df.groupby('Quadrant')['R'].mean().sort_values(ascending=False)
    
    print("\nRun production by team strategy (quadrant):")
    for quadrant, runs in quadrant_runs.items():
        print(f"- {quadrant}: {runs:.1f} runs per season")

# Save final insights
with open('reports/key_findings.txt', 'w') as f:
    f.write("KEY FINDINGS FROM MLB STATCAST ANALYSIS (2015-2024)\n")
    f.write("==================================================\n\n")
    
    f.write("1. Most Predictive Metrics for Run Production:\n")
    if 'correlations' in locals():
        for metric, corr in correlations[:5]:
            f.write(f"   - {metric}: r = {corr:.3f}\n")
    
    f.write("\n2. Best Regression Model:\n")
    if combined_results:
        best_model = max(combined_results.items(), key=lambda x: x[1]['adj_r_squared'])
        f.write(f"   - Type: {best_model[0]}\n")
        f.write(f"   - R-squared: {best_model[1]['r_squared']:.3f}\n")
        f.write(f"   - Adjusted R-squared: {best_model[1]['adj_r_squared']:.3f}\n")
        f.write(f"   - Key predictors: {', '.join(best_model[1]['predictors'])}\n")
    
    f.write("\n3. Evolution of the Game (2015-2024):\n")
    if len(combined_df['Year'].unique()) > 1:
        for metric in available_metrics[:5]:
            if metric in combined_df.columns:
                yearly_means = combined_df.groupby('Year')[metric].mean()
                start_val = yearly_means.iloc[0]
                end_val = yearly_means.iloc[-1]
                change_pct = 100 * (end_val - start_val) / start_val
                
                f.write(f"   - {metric}: {change_pct:+.1f}% change\n")
    
    f.write("\n4. Most Effective Team Strategy:\n")
    if 'Contact_Score' in combined_df.columns and 'Power_Score' in combined_df.columns:
        top_quadrant = quadrant_runs.index[0]
        f.write(f"   - {top_quadrant} approach produced the most runs on average\n")
        
        # Get example of top team with this approach
        top_team = combined_df[combined_df['Quadrant'] == top_quadrant].sort_values('R', ascending=False).iloc[0]
        f.write(f"   - Example: {top_team['Team']} ({top_team['Year']}) with {top_team['R']} runs\n")
    
    f.write("\n5. Recommendations for Building Offensive Strategy:\n")
    if combined_results:
        f.write("   Based on regression analysis, teams should prioritize:\n")
        for model_type, results in combined_results.items():
            significant_predictors = [(var, results['model'].params[var]) 
                                    for var in results['model'].params.index[1:] 
                                    if results['model'].pvalues[var] < 0.05]
            significant_predictors.sort(key=lambda x: abs(x[1]), reverse=True)
            
            if significant_predictors:
                top_predictor = significant_predictors[0]
                f.write(f"   - {top_predictor[0]}: Most significant predictor in {model_type} model\n")

print("\nAnalysis complete! Check the 'figures' and 'reports' folders for visualizations and detailed results.")



SUMMARY OF FINDINGS

1. Correlation Analysis:

Strongest correlations with run production (All Years):
- BB: r = 0.890
- HR: r = 0.875
- AVG: r = 0.402
- SLG: r = 0.374
- OBP: r = 0.324

2. Regression Models:

Traditional Model (All Years):
- R-squared: 0.955
- Adjusted R-squared: 0.955
- Key predictors:
  * AVG: coefficient = 5972.642, p-value = 0.000
  * OBP: coefficient = -2486.658, p-value = 0.000
  * SLG: coefficient = -866.524, p-value = 0.002

Statcast Model (All Years):
- R-squared: 0.091
- Adjusted R-squared: 0.079
- Key predictors:
  * EV: coefficient = 99.840, p-value = 0.000

Combined Model (All Years):
- R-squared: 0.213
- Adjusted R-squared: 0.197
- Key predictors:
  * AVG: coefficient = 9908.863, p-value = 0.004
  * EV: coefficient = 26.887, p-value = 0.013
  * LA: coefficient = 12.178, p-value = 0.031

3. Evolution of Metrics:
- AVG: -4.4% change from 2015 to 2024
- OBP: -1.5% change from 2015 to 2024
- SLG: -1.4% change from 2015 to 2024
- HR: +11.1% change from 2015